# Data Collection for MLB Series Project
See (http://52.14.213.53:8501/) for final Dashboard

#### Metadata:
* Author: MacKenzye Leroy
* Contact: mackenzye-leroy.com

#### Original Question:
* Is regular season series record indicative of postseason success?

#### Goal of this Notebook: 
* Collect, clean, and save data to asnwer question above.
<br>
<br>

Note: Data collection and especially cleaning is an extremely iterative process. This notebook does not cover all of the cleaning I did to make the final Dashboard listed above. Further, most of the cleaing that is presented was discovered teh hard way-by finding something off in my results. This Notebook does show the overall logic and a lot of the major cleaning needed. If you have any firther questions, naviagte to my website (listed above) and feel free to reach out. 

## Import

In [51]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup, NavigableString
#import sys
#import matplotlib.pyplot as plt
#import seaborn as sns
import os

In [52]:
#os.getcwd()

## Scraper
My first goal is to write a scraper to collect the results of all baseball games played since 1900. I need the date. To collect the data, I am taking advantage of the fact that Baseball-Reference.com has a page for every season in MLB history and they are all strictured similarly

<br>
Example: https://www.baseball-reference.com/leagues/majors/2021-schedule.shtml

In [53]:
#Headers for requests
headers = {'user-agent': 'MLeroy, UVA MSDS Student (zuf9mc@virginia.edu)'}

#years of interest
year_list = range(1900, 2022, 1)

#set up url list
urllist = [f"https://www.baseball-reference.com/leagues/majors/{str(x)}-schedule.shtml" for x in year_list]

#check list
urllist[:10]

['https://www.baseball-reference.com/leagues/majors/1900-schedule.shtml',
 'https://www.baseball-reference.com/leagues/majors/1901-schedule.shtml',
 'https://www.baseball-reference.com/leagues/majors/1902-schedule.shtml',
 'https://www.baseball-reference.com/leagues/majors/1903-schedule.shtml',
 'https://www.baseball-reference.com/leagues/majors/1904-schedule.shtml',
 'https://www.baseball-reference.com/leagues/majors/1905-schedule.shtml',
 'https://www.baseball-reference.com/leagues/majors/1906-schedule.shtml',
 'https://www.baseball-reference.com/leagues/majors/1907-schedule.shtml',
 'https://www.baseball-reference.com/leagues/majors/1908-schedule.shtml',
 'https://www.baseball-reference.com/leagues/majors/1909-schedule.shtml']

Using requests, I will visit each url in the list and with beautiful soup I will parse the text. I am interested in which teams played in each game, how many runs they each scored, the data of the game, and for each page I visit, I want to save the postseason starting date (to differentiate postseason versus regular season games later)

In [4]:
#Establish empty dataframes for game results and post season start dates
gameDF = pd.DataFrame()
PostSeasonMarkerDF = pd.DataFrame(columns = ['PostSeasonStartDate'])

#loop thorugh each url
for x in range(len(urllist)):
    #request and parse each page
    r = requests.get(urllist[x], headers = headers)
    parsed = BeautifulSoup(r.text, 'html')
       
    ###collect postseason date info###    
    
    #No Postseasons in 1900-1903 or 1994
    if x > 4 and x != 94:
        #get first date under postseason subheader
        postseasonmarker = parsed.find_all('h2')[1].find_next('h3').text
        df_length = len(PostSeasonMarkerDF)
        PostSeasonMarkerDF.loc[df_length] = postseasonmarker
     
    ###collect game info###
    
    game_list = parsed.find_all("p", "game")
    #print to check status
    print(game_list[0].a)
    
    #set up list to compile info
    home_team_list = []
    away_team_list = []
    home_team_score_list = []
    away_team_score_list = []
    date_list = []
    
    #for each game on page, extract info and append to relevant list
    for y in range(len(game_list)):
        gamex = game_list[y]
        date = gamex.parent.h3.text
        away_team = gamex.a.text
        away_score = gamex.a.next_sibling
        home_team = gamex.a.find_next('a')
        home_score = gamex.a.find_next('a').next_sibling
        home_team_list.append(home_team)
        away_team_list.append(away_team)
        home_team_score_list.append(home_score)
        away_team_score_list.append(away_score)
        date_list.append(date)
    
    #turn lists into data frame
    d = {'Home_Team':home_team_list,
            'Home_Team_Score':home_team_score_list,
            'Away_Team':away_team_list, 
            'Away_Team_Score':away_team_score_list, 
            'Date': date_list}
    
    #append to dataframe
    yearDF = pd.DataFrame(d)
    gameDF = gameDF.append(yearDF)


<a href="/teams/PHI/1900.shtml">Philadelphia Phillies</a>
<a href="/teams/BRO/1901.shtml">Brooklyn Superbas</a>
<a href="/teams/BSN/1902.shtml">Boston Beaneaters</a>
<a href="/teams/PIT/1903.shtml">Pittsburgh Pirates</a>
<a href="/teams/NYG/1904.shtml">New York Giants</a>
<a href="/teams/PHI/1905.shtml">Philadelphia Phillies</a>
<a href="/teams/BSN/1906.shtml">Boston Beaneaters</a>
<a href="/teams/STL/1907.shtml">St. Louis Cardinals</a>
<a href="/teams/WSH/1908.shtml">Washington Senators</a>
<a href="/teams/BOS/1909.shtml">Boston Red Sox</a>
<a href="/teams/NYG/1910.shtml">New York Giants</a>
<a href="/teams/BRO/1911.shtml">Brooklyn Dodgers</a>
<a href="/teams/NYG/1912.shtml">New York Giants</a>
<a href="/teams/PHI/1913.shtml">Philadelphia Phillies</a>
<a href="/teams/BUF/1914.shtml">Buffalo Buffeds</a>
<a href="/teams/NEW/1915.shtml">Newark Pepper</a>
<a href="/teams/PHA/1916.shtml">Philadelphia Athletics</a>
<a href="/teams/PHI/1917.shtml">Philadelphia Phillies</a>
<a href="/teams/PH

In [5]:
#Check results
gameDF.sample(20)

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date
1831,[Montreal Expos],\n (0)\n,New York Mets,\n (6),"Saturday, September 10, 1988"
1029,[Los Angeles Dodgers],\n (2),Milwaukee Brewers,\n (1)\n @\n,"Sunday, June 19, 2016"
338,[St. Louis Cardinals],\n (4),Pittsburgh Pirates,\n (3)\n @\n,"Friday, June 8, 1945"
739,[Philadelphia Phillies],\n (2)\n,Pittsburgh Pirates,\n (9),"Saturday, June 12, 1982"
836,[Chicago Orphans],\n (8),Brooklyn Superbas,\n (5)\n @\n,"Sunday, August 24, 1902"
562,[Chicago White Sox],\n (14),Boston Red Sox,\n (9)\n @\n,"Sunday, July 13, 1919"
1073,[New York Yankees],\n (6),Detroit Tigers,\n (5)\n @\n,"Thursday, July 6, 1989"
863,[Boston Red Sox],\n (8),Detroit Tigers,\n (1)\n @\n,"Thursday, June 7, 2001"
924,[Boston Beaneaters],\n (3),Chicago Cubs,\n (1)\n @\n,"Wednesday, August 30, 1905"
956,[Texas Rangers],\n (10),Oakland Athletics,\n (5)\n @\n,"Saturday, June 8, 2019"


The dataframe below will be used later to distinguish between postseason and regular season games as MLB does not allow any postseason games to start until regular season is completely wrapped up:

In [6]:
#Convert to date time, and save a column for the season that corresponds to each date
PostSeasonMarkerDF['PostSeasonStartDate'] = pd.to_datetime(PostSeasonMarkerDF['PostSeasonStartDate'])
PostSeasonMarkerDF['Season'] = pd.DatetimeIndex(PostSeasonMarkerDF['PostSeasonStartDate']).year
PostSeasonMarkerDF = PostSeasonMarkerDF.set_index('Season')
PostSeasonMarkerDF


,PostSeasonStartDate
Season,
1905,1905-10-09
1906,1906-10-09
1907,1907-10-08
1908,1908-10-10
1909,1909-10-08
...,...
2017,2017-10-03
2018,2018-10-02
2019,2019-10-01


## Initial Cleaning
We need to clean up the game results dataframe to be useful

In [7]:
gameDF_clean = gameDF
gameDF_clean

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date
0,[Boston Beaneaters],\n (17)\n \n,Philadelphia Phillies,\n (19),"Thursday, April 19, 1900"
1,[Cincinnati Reds],\n (10)\n \n,Chicago Orphans,\n (13),"Thursday, April 19, 1900"
2,[New York Giants],\n (2)\n \n,Brooklyn Superbas,\n (3),"Thursday, April 19, 1900"
3,[St. Louis Cardinals],\n (3),Pittsburgh Pirates,\n (0)\n @\n,"Thursday, April 19, 1900"
4,[New York Giants],\n (12),Brooklyn Superbas,\n (8)\n @\n,"Friday, April 20, 1900"
...,...,...,...,...,...
2461,[Houston Astros],\n (7),Atlanta Braves,\n (2)\n @\n,"Wednesday, October 27, 2021"
2462,[Atlanta Braves],\n (2),Houston Astros,\n (0)\n @\n,"Friday, October 29, 2021"
2463,[Atlanta Braves],\n (3),Houston Astros,\n (2)\n @\n,"Saturday, October 30, 2021"
2464,[Atlanta Braves],\n (5)\n,Houston Astros,\n (9),"Sunday, October 31, 2021"


In [8]:
gameDF_clean['Home_Team'].iloc[0]

<a href="/teams/BSN/1900.shtml">Boston Beaneaters</a>

In [9]:
#Right now, Home_Team is saved as an html tag-need to convert to a string
gameDF_clean['Home_Team'] = gameDF_clean.Home_Team.astype(str)

In [10]:
gameDF_clean

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date
0,"<a href=""/teams/BSN/1900.shtml"">Boston Beaneat...",\n (17)\n \n,Philadelphia Phillies,\n (19),"Thursday, April 19, 1900"
1,"<a href=""/teams/CIN/1900.shtml"">Cincinnati Red...",\n (10)\n \n,Chicago Orphans,\n (13),"Thursday, April 19, 1900"
2,"<a href=""/teams/NYG/1900.shtml"">New York Giant...",\n (2)\n \n,Brooklyn Superbas,\n (3),"Thursday, April 19, 1900"
3,"<a href=""/teams/STL/1900.shtml"">St. Louis Card...",\n (3),Pittsburgh Pirates,\n (0)\n @\n,"Thursday, April 19, 1900"
4,"<a href=""/teams/NYG/1900.shtml"">New York Giant...",\n (12),Brooklyn Superbas,\n (8)\n @\n,"Friday, April 20, 1900"
...,...,...,...,...,...
2461,"<a href=""/teams/HOU/2021.shtml"">Houston Astros...",\n (7),Atlanta Braves,\n (2)\n @\n,"Wednesday, October 27, 2021"
2462,"<a href=""/teams/ATL/2021.shtml"">Atlanta Braves...",\n (2),Houston Astros,\n (0)\n @\n,"Friday, October 29, 2021"
2463,"<a href=""/teams/ATL/2021.shtml"">Atlanta Braves...",\n (3),Houston Astros,\n (2)\n @\n,"Saturday, October 30, 2021"
2464,"<a href=""/teams/ATL/2021.shtml"">Atlanta Braves...",\n (5)\n,Houston Astros,\n (9),"Sunday, October 31, 2021"


In [11]:
gameDF_clean['Home_Team'].iloc[0]

'<a href="/teams/BSN/1900.shtml">Boston Beaneaters</a>'

In [12]:
#Strip away irrelevant text in front of home team string
gameDF_clean['Home_Team'] = gameDF_clean.Home_Team.str[32:]

In [13]:
gameDF_clean.head(5)

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date
0,Boston Beaneaters</a>,\n (17)\n \n,Philadelphia Phillies,\n (19),"Thursday, April 19, 1900"
1,Cincinnati Reds</a>,\n (10)\n \n,Chicago Orphans,\n (13),"Thursday, April 19, 1900"
2,New York Giants</a>,\n (2)\n \n,Brooklyn Superbas,\n (3),"Thursday, April 19, 1900"
3,St. Louis Cardinals</a>,\n (3),Pittsburgh Pirates,\n (0)\n @\n,"Thursday, April 19, 1900"
4,New York Giants</a>,\n (12),Brooklyn Superbas,\n (8)\n @\n,"Friday, April 20, 1900"


In [14]:
#Strip away irrelevant text in back of home team string
gameDF_clean['Home_Team'] = gameDF_clean.Home_Team.str[:-4]

In [15]:
#check results
gameDF_clean.sample(10)

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date
358,Colorado Rockies,\n (5)\n,St. Louis Cardinals,\n (6),"Wednesday, May 4, 1994"
1112,Pittsburgh Pirates,\n (8),Philadelphia Phillies,\n (6)\n @\n,"Wednesday, September 12, 1951"
211,Detroit Tigers,\n (5)\n,Texas Rangers,\n (10),"Tuesday, April 29, 1980"
2228,Philadelphia Phillies,\n (4)\n,Miami Marlins,\n (5),"Sunday, September 14, 2014"
214,Boston Doves,\n (2)\n,Chicago Cubs,\n (6),"Thursday, May 20, 1909"
837,New York Giants,\n (5),Pittsburgh Pirates,\n (1)\n @\n,"Wednesday, August 11, 1920"
1194,Minnesota Twins,\n (1)\n,Boston Red Sox,\n (2),"Saturday, September 2, 1961"
613,Brooklyn Robins,\n (0)\n,Chicago Cubs,\n (1),"Sunday, July 10, 1927"
1215,Detroit Tigers,\n (4),Chicago White Sox,\n (2)\n @\n,"Saturday, July 21, 1979"
1149,Boston Red Sox,\n (9),Chicago White Sox,\n (3)\n @\n,"Monday, September 14, 1959"


In [16]:
#Strip all but int values for home team and away team score
gameDF_clean['Home_Team_Score'] = gameDF_clean.Home_Team_Score.str.extract('(\d+)')
gameDF_clean['Away_Team_Score'] = gameDF_clean.Away_Team_Score.str.extract('(\d+)')

In [17]:
#Check results
gameDF_clean.sample(10)

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date
75,Milwaukee Braves,5,St. Louis Cardinals,4,"Saturday, April 23, 1955"
556,Los Angeles Dodgers,8,Houston Astros,2,"Tuesday, May 28, 1991"
1092,New York Giants,4,Cincinnati Reds,2,"Wednesday, September 9, 1936"
419,Chicago White Sox,2,Seattle Mariners,9,"Friday, May 2, 2003"
767,Pittsburgh Pirates,9,Boston Braves,8,"Thursday, July 31, 1941"
809,San Diego Padres,8,Colorado Rockies,5,"Friday, June 2, 2017"
563,San Francisco Giants,8,Los Angeles Dodgers,4,"Monday, May 15, 2017"
2283,Oakland Athletics,12,Texas Rangers,3,"Thursday, September 24, 2009"
1015,Cleveland Indians,9,Detroit Tigers,11,"Thursday, September 3, 1925"
230,Houston Colt .45s,6,Los Angeles Dodgers,9,"Tuesday, May 8, 1962"


In [18]:
#Add column for winning team
gameDF_clean['Winner'] = np.where(gameDF['Home_Team_Score'] > gameDF['Away_Team_Score'] , gameDF['Home_Team'], gameDF['Away_Team'])

In [19]:
#replace "D'Backs" with 'Diamondbacks to avoid problmes with data later (Apostrphes are trouble)
gameDF_clean["Home_Team"].replace({"Arizona D'Backs": "Arizona Diamondbacks"}, inplace=True)
gameDF_clean["Away_Team"].replace({"Arizona D'Backs": "Arizona Diamondbacks"}, inplace=True)

In [20]:
#check no Na values

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date,Winner


In [21]:
#check team values
gameDF_clean["Home_Team"].value_counts()

St. Louis Cardinals      9647
Pittsburgh Pirates       9556
Philadelphia Phillies    9494
Detroit Tigers           9487
Chicago White Sox        9440
                         ... 
Buffalo Blues              77
Boston Rustlers            75
Cleveland Blues            69
Cleveland Bronchos         65
                            5
Name: Home_Team, Length: 71, dtype: int64

In [ ]:
#Check for NaN's

In [22]:
gameDF_clean.isnull().sum()

Home_Team           0
Home_Team_Score    77
Away_Team           0
Away_Team_Score    77
Date                0
Winner              0
dtype: int64

In [23]:
gameDF_clean[gameDF_clean['Home_Team_Score'].isnull()]

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date,Winner
487,Brooklyn Superbas,NaN,St. Louis Cardinals,NaN,"Wednesday, September 19, 1900",St. Louis Cardinals
524,Chicago Orphans,NaN,St. Louis Cardinals,NaN,"Saturday, September 29, 1900",St. Louis Cardinals
569,,NaN,,NaN,Today's Games,
1109,,NaN,,NaN,Today's Games,
70,Chicago Orphans,NaN,Cincinnati Reds,NaN,"Thursday, May 1, 1902",Cincinnati Reds
...,...,...,...,...,...,...
221,Baltimore Orioles,NaN,Kansas City Athletics,NaN,"Sunday, May 19, 1957",Kansas City Athletics
1607,New York Mets,NaN,Philadelphia Phillies,NaN,"Saturday, October 2, 1965",Philadelphia Phillies
1132,Chicago White Sox,NaN,Detroit Tigers,NaN,"Thursday, July 12, 1979",Detroit Tigers
1895,St. Louis Cardinals,NaN,Pittsburgh Pirates,NaN,"Wednesday, September 13, 1989",Pittsburgh Pirates


Note: Most of these are delays or ties. We will leave in for now as we can clean them up later if we choose to. Columns with 'Today's Game' however are an issue and will be dropped

In [24]:
gameDF_clean[gameDF_clean['Date'] == "Today's Games"]

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date,Winner
569,,NaN,,NaN,Today's Games,
1109,,NaN,,NaN,Today's Games,
1115,,NaN,,NaN,Today's Games,
1248,,NaN,,NaN,Today's Games,
1600,,NaN,,NaN,Today's Games,


In [25]:
gameDF_clean = gameDF_clean[gameDF_clean['Date'] != "Today's Games"]

In [56]:
#Convert Date to date time 
gameDF_clean['Date'] = pd.to_datetime(gameDF_clean["Date"])
gameDF_clean['Season'] = pd.DatetimeIndex(gameDF_clean['Date']).year

<ipython-input-56-b2fe8f72179d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gameDF_clean['Date'] = pd.to_datetime(gameDF_clean["Date"])
<ipython-input-56-b2fe8f72179d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gameDF_clean['Season'] = pd.DatetimeIndex(gameDF_clean['Date']).year


In [57]:
gameDF_clean.sample(10)

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date,Winner,Season
781,Milwaukee Brewers,2,Seattle Mariners,1,1993-06-08,Milwaukee Brewers,1993
395,Pittsburgh Pirates,5,Boston Braves,0,1952-06-11,Pittsburgh Pirates,1952
2108,New York Mets,1,Washington Nationals,5,2012-09-10,Washington Nationals,2012
444,New York Yankees,2,Baltimore Orioles,6,2015-05-09,Baltimore Orioles,2015
33,Philadelphia Athletics,1,Boston Red Sox,0,1911-04-17,Philadelphia Athletics,1911
148,Detroit Tigers,7,Boston Red Sox,15,1925-05-08,Detroit Tigers,1925
1079,Minnesota Twins,3,Boston Red Sox,4,1988-07-07,Boston Red Sox,1988
493,Boston Doves,2,Philadelphia Phillies,9,1908-06-29,Philadelphia Phillies,1908
1524,Chicago White Sox,8,Boston Red Sox,3,1963-09-18,Chicago White Sox,1963
388,Boston Red Sox,7,Detroit Tigers,12,1936-06-08,Boston Red Sox,1936


In [58]:
gameDF_clean.tail(10)

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date,Winner,Season
2456,Los Angeles Dodgers,2,Atlanta Braves,9,2021-10-20,Atlanta Braves,2021
2457,Los Angeles Dodgers,11,Atlanta Braves,2,2021-10-21,Atlanta Braves,2021
2458,Houston Astros,5,Boston Red Sox,0,2021-10-22,Houston Astros,2021
2459,Atlanta Braves,4,Los Angeles Dodgers,2,2021-10-23,Atlanta Braves,2021
2460,Houston Astros,2,Atlanta Braves,6,2021-10-26,Atlanta Braves,2021
2461,Houston Astros,7,Atlanta Braves,2,2021-10-27,Houston Astros,2021
2462,Atlanta Braves,2,Houston Astros,0,2021-10-29,Atlanta Braves,2021
2463,Atlanta Braves,3,Houston Astros,2,2021-10-30,Atlanta Braves,2021
2464,Atlanta Braves,5,Houston Astros,9,2021-10-31,Houston Astros,2021
2465,Houston Astros,0,Atlanta Braves,7,2021-11-02,Atlanta Braves,2021


In [29]:

gameDF_clean

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date,Winner,Season
0,Boston Beaneaters,17,Philadelphia Phillies,19,1900-04-19,Philadelphia Phillies,1900
1,Cincinnati Reds,10,Chicago Orphans,13,1900-04-19,Chicago Orphans,1900
2,New York Giants,2,Brooklyn Superbas,3,1900-04-19,Brooklyn Superbas,1900
3,St. Louis Cardinals,3,Pittsburgh Pirates,0,1900-04-19,St. Louis Cardinals,1900
4,New York Giants,12,Brooklyn Superbas,8,1900-04-20,Brooklyn Superbas,1900
...,...,...,...,...,...,...,...
2461,Houston Astros,7,Atlanta Braves,2,2021-10-27,Houston Astros,2021
2462,Atlanta Braves,2,Houston Astros,0,2021-10-29,Atlanta Braves,2021
2463,Atlanta Braves,3,Houston Astros,2,2021-10-30,Atlanta Braves,2021
2464,Atlanta Braves,5,Houston Astros,9,2021-10-31,Houston Astros,2021


In [30]:
#Add Winner Column
gameDF_clean['Winner'] = np.where(gameDF_clean['Home_Team_Score'] > gameDF_clean['Away_Team_Score'] , gameDF_clean['Home_Team'], gameDF_clean['Away_Team'])


<ipython-input-30-b4dad206c571>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gameDF_clean['Winner'] = np.where(gameDF_clean['Home_Team_Score'] > gameDF_clean['Away_Team_Score'] , gameDF_clean['Home_Team'], gameDF_clean['Away_Team'])


In [31]:
gameDF_clean

,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,Date,Winner,Season
0,Boston Beaneaters,17,Philadelphia Phillies,19,1900-04-19,Philadelphia Phillies,1900
1,Cincinnati Reds,10,Chicago Orphans,13,1900-04-19,Chicago Orphans,1900
2,New York Giants,2,Brooklyn Superbas,3,1900-04-19,Brooklyn Superbas,1900
3,St. Louis Cardinals,3,Pittsburgh Pirates,0,1900-04-19,St. Louis Cardinals,1900
4,New York Giants,12,Brooklyn Superbas,8,1900-04-20,Brooklyn Superbas,1900
...,...,...,...,...,...,...,...
2461,Houston Astros,7,Atlanta Braves,2,2021-10-27,Houston Astros,2021
2462,Atlanta Braves,2,Houston Astros,0,2021-10-29,Atlanta Braves,2021
2463,Atlanta Braves,3,Houston Astros,2,2021-10-30,Atlanta Braves,2021
2464,Atlanta Braves,5,Houston Astros,9,2021-10-31,Houston Astros,2021


## Function for Calculting Series Results
The next step is writing a function given a team and year, can return how many games and series that team played as well as how many of each they won, tied and lost. Below is that function. 

In [32]:
def OneTeamFunction(Team, Year):
    
    #Filter dataframe for only the team of interest
    query1 = f"Home_Team == '{Team}' | Away_Team== '{Team}'"
    df1 = gameDF_clean.query(query1)
    df1 = df1.reset_index()
    
    #Filter dataframe for only the team of interest
    query2 = f"Season == {Year}"
    df1 = df1.query(query2)
    
    #Filter out postseason results
    noPostSeasonList = [1900, 1901, 1902, 1903, 1904, 1994] #No postseason in these years
    if Year not in noPostSeasonList:
        postSeasonCheck = PostSeasonMarkerDF.loc[Year][0]
        df1 = df1[df1.Date < postSeasonCheck]

    #Creat a dataframe of the game results shifted
    df1_shift = df1.shift()
    df1_shift = df1_shift.rename(columns ={'Home_Team':'Last_Home_Team',
                               'Away_Team':'Last_Away_Team',
                              'Winner': 'Last_Winner'})
    
    #Only need Last_Away_Team, Last_Home_Team, and Last_winner
    df1_shift = df1_shift.drop(columns =['index','Home_Team_Score', 'Away_Team_Score', 'Date', 'Season'])
    
    #concat two dataframes together. If Home_Team != Last_Home_Team & Away_Team != Last_Away_Team, we know we have a new series 
    df1 = pd.concat([df1, df1_shift], axis = 1)
    #add columne to track these
    df1['New_Series'] = np.where(((df1['Home_Team'] == df1['Last_Home_Team'])
                                        & (df1['Away_Team'] == df1['Last_Away_Team'])), False, True)
    
    #Take advantage of Boolean True = 1 and do cumulative sum to track series number
    df1['Series_Number'] = df1.New_Series.cumsum()
    
    #create column to see if team of interest won or lost
    df1['Team_Winner'] = np.where(((df1['Winner'] == Team)), True, False)
    
    #Get total wins with cumulative sum (again taking advantage of boolean True = 1)
    df1['Team_Wins'] = df1.Team_Winner.cumsum()
    
    #Now, group by series number and sum the number of wins-save in its own df
    df2 = df1.groupby(by=["Series_Number"]).agg({'Series_Number': ['count'], 'Team_Winner': ['sum']})
    df2 = pd.DataFrame(df2.to_records())
    df2 = df2.rename(columns = {"('Series_Number', 'count')": "GamesInSeries", "('Team_Winner', 'sum')": "TeamWins"})
    
    #Now check if team won a majority of games in series. If so, True 
    df2["TeamSeriesWinner"] = np.where(((df2['TeamWins'] > (df2['GamesInSeries']/2))), True, False)
    
    #If even label series a tie
    df2["SeriesTie"] = np.where(((df2['TeamWins'] == (df2['GamesInSeries']/2))), True, False)
    
    #cumulative sum each column
    df2["SeriesWins"] = df2.TeamSeriesWinner.cumsum()
    df2["SeriesTies"] = df2.SeriesTie.cumsum()
    
    #compile totals
    NumberOfSeries = df2.iloc[-1].Series_Number
    SeriesWon = df2.iloc[-1].SeriesWins
    SeriesTies = df2.iloc[-1].SeriesTies
    SeriesLoss = NumberOfSeries - SeriesWon - SeriesTies
    NumberOfWins = df1.iloc[-1].Team_Wins
    NumberOfGames = len(df1)
    NumberOfLosses = NumberOfGames - NumberOfWins
     
    #Insert totals into a list and return
    YearList = [Team, Year, NumberOfGames, NumberOfWins, NumberOfLosses, NumberOfSeries, SeriesWon, SeriesLoss, SeriesTies]
    return YearList

tests:

In [33]:
OneTeamFunction('New York Mets', 1986)

['New York Mets', 1986, 162, 101, 61, 54, 30, 19, 5]

In [34]:
OneTeamFunction('New York Mets', 1969)

['New York Mets', 1969, 162, 100, 62, 54, 29, 13, 12]

In [35]:
OneTeamFunction('Atlanta Braves', 2021)

['Atlanta Braves', 2021, 161, 89, 72, 52, 27, 18, 7]

## Compiling all time results

Now that we have a function for detemining the win/loss record and win/tie/loss series record for any team on a season over season basis, we can run the function on all team and year combos in dataframe and save results

In [36]:
#First, get all unique team/year combos
TeamsAndYearsFull = pd.DataFrame(gameDF_clean.groupby(by=["Home_Team", "Season"]).agg({'Season': ['max']}).to_records())
TeamsAndYearsFull = TeamsAndYearsFull[["Home_Team", "Season"]]

In [37]:
TeamsAndYearsFull

,Home_Team,Season
0,Anaheim Angels,1997
1,Anaheim Angels,1998
2,Anaheim Angels,1999
3,Anaheim Angels,2000
4,Anaheim Angels,2001
...,...,...
2605,Washington Senators,1967
2606,Washington Senators,1968
2607,Washington Senators,1969
2608,Washington Senators,1970


Now we can loop through each team/year combo and run our function

In [38]:
#initiate a df for compiling results
MasterResultsDF = pd.DataFrame(columns =["Team", "Year", 
                                "NumberOfGames", "Wins", "Losses", 
                                "NumberOfSeries", "SeriesWins", "SeriesLosses", "SeriesTies"])
#loop through each combo, run function and save result to df
for x in range(len(TeamsAndYearsFull)):
    team = TeamsAndYearsFull.iloc[x][0]
    year = TeamsAndYearsFull.iloc[x][1]
    currentyear = OneTeamFunction(team, year)
    df_length = len(MasterResultsDF)
    MasterResultsDF.loc[df_length] = currentyear

In [39]:
#Check
MasterResultsDF

,Team,Year,NumberOfGames,Wins,Losses,NumberOfSeries,SeriesWins,SeriesLosses,SeriesTies
0,Anaheim Angels,1997,162,84,78,60,24,22,14
1,Anaheim Angels,1998,162,74,88,58,20,25,13
2,Anaheim Angels,1999,162,71,91,52,18,29,5
3,Anaheim Angels,2000,162,83,79,52,21,21,10
4,Anaheim Angels,2001,162,71,91,53,22,27,4
...,...,...,...,...,...,...,...,...,...
2605,Washington Senators,1967,161,80,81,55,21,22,12
2606,Washington Senators,1968,161,66,95,55,17,28,10
2607,Washington Senators,1969,162,81,81,54,22,23,9
2608,Washington Senators,1970,162,74,88,55,22,27,6


Great, we now have the results for all team name/year results. Unfortuntately, some teams have had several names throughout their history (think of the darned Angels) and even worse some franchises have each used the same name at different points in their histories (look up Washington Senators history). Need to add a unique identifer (FranID) for franchises. 

In [40]:
teams_ID_dictionary= {'Tampa Bay Devil Rays':'TBD', 'Tampa Bay Rays': 'TBD', 'Florida Marlins': 'FLA', 'Miami Marlins':'FLA',
                      'Montreal Expos':'WSN', 'Washington Nationals':'WSN',  'Seattle Pilots':'MIL', 'Milwaukee Brewers':'MIL',
                      'Houston Colt .45s':'HOU', 'Houston Astros':'HOU', 'Washington Senators':'MIN', 'Minnesota Twins': 'MIN',
                      'California Angels':'ANA','Anaheim Angels':'ANA', 'LA Angels of Anaheim':'ANA', 'Los Angeles Angels':'ANA', 
                      'Philadelphia Athletics':'OAK', 'Kansas City Athletics':'OAK', 'Oakland Athletics':'OAK', 
                      'Baltimore Orioles':'BAL','St. Louis Browns':'BAL', 'Cleveland Indians':'CLE', 'Cleveland Naps':'CLE', 'Cleveland Blues':'CLE',
                      'Boston Red Sox':'BOS', 'Boston Americans':'BOS', 'Cincinnati Reds':'CIN','Cincinnati Redlegs':'CIN',
                      'New York Yankees':'NYY', 'New York Highlanders':'NYY', 'Chicago Cubs':'CHC', 'Chicago Orphans':'CHC',
                      'Los Angeles Dodgers':'LAD', 'Brooklyn Superbas':'LAD','Brooklyn Dodgers':'LAD', 'Brooklyn Robins':'LAD',
                      'San Francisco Giants':'SFG', 'New York Giants':'SFG', 'New York Mets':'NYM', 'Atlanta Braves':'ATL', 
                      'Milwaukee Braves':'ATL', 'Boston Braves':'ATL', 'Boston Doves':'ATL', 'Boston Beaneaters':'ATL', 
                      'Boston Bees':'ATL', 'Boston Rustlers':'ATL',  'Pittsburgh Pirates':'PIT', 'Philadelphia Phillies':'PHI',
                      'Chicago White Sox':'CHW', 'Detroit Tigers':'DET', 'Texas Rangers':'TEX', 'Kansas City Royals':'KCR', 
                      'San Diego Padres':'SDP', 'Arizona Diamondbacks':'ARI', 'Seattle Mariners':'SEA',
                      'Toronto Blue Jays':'TOR', 'Colorado Rockies':'COL', 'St. Louis Cardinals':'STL' ,
                     'Baltimore Terrapins':'FedBAL', 'St. Louis Terriers':'FedSTL','Brooklyn Tip-Tops':'FedBRK',
                     'Pittsburgh Rebels': 'FedPIT', 'Kansas City Packers':'FedKCP', 'Indianapolis Hoosiers':'FedINDNEW',
                      'Newark Pepper':'FedINDNEW', 'Buffalo Buffeds':'FedBUF', 'Buffalo Blues':'FedBUF', 'Chicago Whales':'FedCHI',
                      'Chicago Chi-Feds':'FedCHI', 'Cleveland Bronchos':'FedCLE'  
                     }

#Note: Results contain data from short-lived Federation League. Tags that start with Fed correspond to those teams

In [41]:
#First, add a placeholder of team name in FranID column
MasterResultsDF['FranID'] = MasterResultsDF['Team']

#replace with FranID per dictionary above
MasterResultsDF['FranID'].replace(teams_ID_dictionary, inplace= True)

#Check results
MasterResultsDF.sample(20)

,Team,Year,NumberOfGames,Wins,Losses,NumberOfSeries,SeriesWins,SeriesLosses,SeriesTies,FranID
671,Chicago White Sox,2020,60,33,27,20,11,7,2,CHW
557,Chicago White Sox,1906,154,93,61,52,28,16,8,CHW
1386,Minnesota Twins,1972,154,82,72,52,27,19,6,MIN
770,Cincinnati Reds,1997,162,85,77,57,21,19,17,CIN
805,Cleveland Indians,1923,153,85,68,45,20,11,14,CLE
2303,St. Louis Cardinals,1924,154,63,91,46,16,22,8,STL
1254,Los Angeles Dodgers,1977,162,93,69,54,29,14,11,LAD
602,Chicago White Sox,1951,155,79,76,55,25,20,10,CHW
2327,St. Louis Cardinals,1948,155,87,68,56,30,19,7,STL
1517,New York Giants,1945,154,86,68,50,26,18,6,SFG


That should fix teams that have cycled through multiple names, but won't do anything about different franchises sharing the same name. For that we'll need to manually fix

In [42]:
#First, fix Washington Senators (Washington Senators could correspond to the modern day Rangers or Twins depending on the year)
MasterResultsDF.loc[(MasterResultsDF['Team'] == 'Washington Senators') & (MasterResultsDF['Year']>1960), 'FranID'] = 'TEX'

In [43]:
#Second fix orioles/yankees
MasterResultsDF.loc[(MasterResultsDF['Team'] == 'Baltimore Orioles') & (MasterResultsDF['Year']<1905), 'FranID'] = 'NYY'

In [44]:
#Check Early Orioles (which should be yankees)
MasterResultsDF.loc[(MasterResultsDF['Team'] == 'Baltimore Orioles') & (MasterResultsDF['Year']<1905)]

,Team,Year,NumberOfGames,Wins,Losses,NumberOfSeries,SeriesWins,SeriesLosses,SeriesTies,FranID
88,Baltimore Orioles,1901,134,64,70,42,11,16,15,NYY
89,Baltimore Orioles,1902,140,57,83,45,13,24,8,NYY


In [59]:
#Check that Senators correspond to MIN until 1960 and to TEX from 1961 on
MasterResultsDF[MasterResultsDF.Team == 'Washington Senators'].tail(15)

,Team,Year,NumberOfGames,Wins,Losses,NumberOfSeries,SeriesWins,SeriesLosses,SeriesTies,FranID
2595,Washington Senators,1957,154,60,94,58,13,33,12,MIN
2596,Washington Senators,1958,156,65,91,57,20,26,11,MIN
2597,Washington Senators,1959,154,65,89,57,15,27,15,MIN
2598,Washington Senators,1960,154,78,76,61,27,21,13,MIN
2599,Washington Senators,1961,161,66,95,54,18,28,8,TEX
2600,Washington Senators,1962,162,66,96,56,12,29,15,TEX
2601,Washington Senators,1963,162,65,97,57,15,34,8,TEX
2602,Washington Senators,1964,162,64,98,54,13,29,12,TEX
2603,Washington Senators,1965,162,74,88,59,19,25,15,TEX
2604,Washington Senators,1966,159,72,87,58,20,25,13,TEX


In [61]:
#Check overall results to see if any outliers-looks good
MasterResultsDF['FranID'].value_counts()

CHC          122
PIT          122
STL          122
SFG          122
PHI          122
ATL          122
LAD          122
CIN          122
CHW          121
OAK          121
MIN          121
BOS          121
NYY          121
DET          121
BAL          120
CLE          120
ANA           61
TEX           61
NYM           60
HOU           60
MIL           54
WSN           53
KCR           53
SDP           53
SEA           45
TOR           45
FLA           29
COL           29
ARI           24
TBD           24
FedBUF         2
FedINDNEW      2
FedSTL         2
FedBRK         2
FedPIT         2
FedBAL         2
FedCHI         2
FedKCP         2
FedCLE         1
Name: FranID, dtype: int64

### Save Results

In [48]:
#MasterResultsDF.to_csv('YearlyResultsMaster.csv')
#gameDF_clean.to_csv('LeagueGameResults.csv', index = False)
#PostSeasonMarkerDF.to_csv('PostSeasonStartDates.csv')